In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float: return np.dot(a, b.reshape(-1, 1)) / (np.linalg.norm(a) * np.linalg.norm(b))

In [3]:
known_embeddings_path: str   = "../input/fv-trf-na384-nofd/known-embeddings/"
unknown_embeddings_path: str = "../input/fv-trf-na384-nofd/unknown-embeddings/"    

In [4]:
swin_b_known_embeddings_path: str   = os.path.join(known_embeddings_path, [filename for filename in sorted(os.listdir(known_embeddings_path)) if "swin_base" in filename][0])
swin_b_unknown_embeddings_path: str = os.path.join(unknown_embeddings_path, [filename for filename in sorted(os.listdir(unknown_embeddings_path)) if "swin_base" in filename][0])

In [5]:
swin_b_known_embeddings: np.ndarray   = np.load(swin_b_known_embeddings_path)
swin_b_unknown_embeddings: np.ndarray = np.load(swin_b_unknown_embeddings_path)

labels: np.ndarray = np.load("../input/fv-trf-na384-nofd/labels.npy")
labels: list = sorted(list(labels))    

In [6]:
predictions: list = []

for i in range(swin_b_unknown_embeddings.shape[0]):
    similarity = -np.inf
    unk_embedding = swin_b_unknown_embeddings[i]
    for j in range(swin_b_known_embeddings.shape[0]):
        k_embedding = swin_b_known_embeddings[j]
        compare_sim = cosine_similarity(unk_embedding.reshape(1, -1), k_embedding.reshape(1, -1))[0][0]
        if compare_sim > similarity:
            label = labels[j]
            similarity = compare_sim
    predictions.append(label)

In [7]:
df = pd.DataFrame(data=None, columns=["id", "label"])

df.id = [i for i in range(len(os.listdir("../input/11-785-f22-hw2p2-verification/verification/unknown_test")))]
df.label = predictions
df.to_csv("submission.csv", index=False)